In [2]:
pip install pyspark


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
  Using cached pip-23.3.1-py3-none-any.whl.metadata (3.5 kB)
Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName('importing data').getOrCreate()

df = spark.read.text("/home/axat/personal/pySpark/data/")
# df.show(truncate=False)

df = df.withColumn("FileName", input_file_name())

df.show(100,truncate=False)
#df.printSchema()
spark.stop()

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------+
|value                                                            

In [54]:
from PIL import Image
import pytesseract

pytesseract.pytesseract.tesseract_cmd = '/home/axat/myenv/bin/pytesseract'

def image_to_text(image_path, lang='eng'):
    image = Image.open(image_path)
    custom_config = f'--psm 6'
    text = pytesseract.image_to_string(image, lang=lang, config=custom_config)
    return text

image_path = "/home/axat/personal/pySpark/data/Screenshot_20231122_105132.png"
text = image_to_text(image_path)
print("Text from image:")
print(text)


TesseractError: (2, 'Usage: pytesseract [-l lang] input_file')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name
from pyspark.sql.types import StructType, StructField, StringType

# Additional imports
from PIL import Image
import glob, PyPDF2, pytesseract

# Create a Spark session
spark = SparkSession.builder.appName('importing data').getOrCreate()

# Define the schema for the DataFrame
schema = StructType([StructField("FileName", StringType(), True),
                     StructField("Text", StringType(), True)])

# Create an empty DataFrame with the specified schema
df = spark.createDataFrame([], schema=schema)

# Function to add rows to the DataFrame
def add_row(filename, text):
    return spark.createDataFrame([(filename, text)], schema=schema)

# Function for text extraction from images
def image_to_text(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Function for text extraction from pdf
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_number in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()
    return text

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text

# Function to extract filename
def extracting_filename(path):
    file_name = path.split("/")[-1].split(".")[0]
    return file_name

# Paths
pdfs = glob.glob("/home/axat/personal/pySpark/data/*.pdf")
images = glob.glob("/home/axat/personal/pySpark/data/*.png")
texts = glob.glob("/home/axat/personal/pySpark/data/*.txt")

# Converting pdfs to text and adding to the DataFrame
for pdf_path in pdfs:
    text = extract_text_from_pdf(pdf_path)
    filename = extracting_filename(pdf_path)
    df = df.union(add_row(filename, text))

# Converting images to text and adding to the DataFrame
for image_path in images:
    text = image_to_text(image_path)
    filename = extracting_filename(image_path)
    df = df.union(add_row(filename, text))

# Reading text from files and adding to the DataFrame
for text_file in texts:
    text = read_text_from_file(text_file)
    filename = extracting_filename(text_file)
    df = df.union(add_row(filename, text))

# Show the DataFrame
df.show(truncate=False)

# Stop the Spark session
spark.stop()


+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------